In [4]:
import numpy as np
import pandas as pd
import re,math
import matplotlib.pyplot as plt
from scipy.optimize import linprog


np.set_printoptions(suppress=True)
data1 = pd.read_csv(r'BCHAIN-MKPRU.csv') # B
data2 = pd.read_csv(r'LBMA-GOLD.csv') # G

# 线性插值填充
data2.interpolate(method='linear', limit_direction='backward', axis=0, inplace = True)

def amplitude(list_1): # 涨幅跌幅计算函数
    x1 = list_1.copy()[1:]
    x2 = list_1.copy()[:-1]
    y = (x1 - x2)/x2
    return y

# 比特币日涨幅
day_b_amp = amplitude(np.array(data1["Value"]))
pd.DataFrame(day_b_amp).T.to_csv("比特币日涨幅.csv")
day_b_amp_b = pd.concat([pd.DataFrame(np.array(data1["Date"])[1:],columns=["Date"]), pd.DataFrame(day_b_amp)],axis=1)


# 黄金日涨幅
day_amp = amplitude(np.array(data2["USD (PM)"]))
pd.DataFrame(day_amp).T.to_csv("黄金日涨幅.csv")
day_g_amp_g = pd.concat([pd.DataFrame(np.array(data2["Date"])[1:],columns=["Date"]), pd.DataFrame(day_amp)],axis=1)





def gold(short_item, long_item,score, a):
    # 周黄金平均值
    count = 1
    temp3 = []
    while True:
        tp = data2.iloc[short_item*(count-1): short_item*count]["USD (PM)"].mean()
        count+= 1
        temp3.append(tp)
        if count >= len(data2)/short_item:
            break
    temp3_df = pd.DataFrame(temp3) # 252周的金价
    # 周平均金价图
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = temp3_df.index
#     y = temp3_df.values
#     plot = ax.plot( x, y, color=dt_hex, linewidth=2, linestyle='-',label='week' )
#     ax.set_xticks( range(0,len(x),10))
#     ax.legend( loc=0, frameon=True )











    # 长期(20天)
    count = 1
    temp4_max = [] # 周高位点
    temp4_min = [] # 周低位点
    while True:
        tp_max = max(temp3[(count-1)*long_item:count*long_item])
        tp_min = min(temp3[(count-1)*long_item:count*long_item])
        temp4_max.append(tp_max)
        temp4_min.append(tp_min)
        count+=1
        if count >= len(temp3)/long_item:
            break
    temp4_max_h = amplitude(np.array(temp4_max)) # 周高位点
    temp4_min_l = amplitude(np.array(temp4_min)) # 周低位点
    #周趋势(同增同减同趋势，不同则未知)
    tptp = []
    for i,j in zip(temp4_max_h, temp4_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
        elif i < 0 and j < 0:
            tptp.append(-1)
        else:
            tptp.append(0)
    tptp_w_g = pd.DataFrame(tptp) # 第二周开始的61周涨跌幅度表 -1跌+1涨
    #周趋势图(长期)
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = tptp_w_g.index
#     y = tptp_w_g.values
#     plot = ax.plot( x, y,"r*", color=dt_hex )
#     ax.set_xticks( range(2,len(x),10))
    # ax.legend( loc=0, frameon=True )
    # plt.show()









    # # (15天)
    # count = 1
    # temp5_max = []
    # temp5_min = []
    # temp5_max_index = []
    # temp5_min_index = []
    # while True:
    #     tp_max = max(np.array(data2["USD (PM)"])[(count-1)*15:count*15])
    #     tp_min = min(np.array(data2["USD (PM)"])[(count-1)*15:count*15])
    #     temp5_max_index.append((count-1)*15+list(np.array(data2["USD (PM)"])[(count-1)*15:count*15]).index(tp_max))
    #     temp5_min_index.append((count-1)*15+list(np.array(data2["USD (PM)"])[(count-1)*15:count*15]).index(tp_min))
    #     temp5_max.append(tp_max)
    #     temp5_min.append(tp_min)
    #     count+=1
    #     if count >= len(data2)/15:
    #         break
    # temp5_max_h = amplitude(np.array(temp5_max))
    # temp5_min_l = amplitude(np.array(temp5_min))
    # #天趋势
    # tptp = []
    # temp5_max_value = []
    # temp5_min_value = []
    # temp5_max_index_real = []
    # temp5_min_index_real = []
    # count = 0
    # for i,j in zip(temp5_max_h, temp5_min_l):
    #     if i > 0 and j > 0:
    #         tptp.append(1)
    #         temp5_max_value.append(np.array(data2.loc[[temp5_max_index[count]],["USD (PM)"]])[0])
    #         temp5_min_value.append(np.array(data2.loc[[temp5_min_index[count]],["USD (PM)"]])[0])
    #         temp5_max_index_real.append(temp5_max_index[count])
    #         temp5_min_index_real.append(temp5_min_index[count])
    #     elif i < 0 and j < 0:
    #         tptp.append(-1)
    #         temp5_max_value.append(np.array(data2.loc[[temp5_max_index[count]],["USD (PM)"]])[0])
    #         temp5_min_value.append(np.array(data2.loc[[temp5_min_index[count]],["USD (PM)"]])[0])
    #         temp5_max_index_real.append(temp5_max_index[count])
    #         temp5_min_index_real.append(temp5_min_index[count])
    #     else:
    #         tptp.append(0)
    #     count += 1
    # tptp_d = pd.DataFrame(tptp)
    # # 点图
    # fig = plt.figure( figsize=(16,4), dpi=100)
    # ax = fig.add_subplot(1,1,1)
    # x = tptp_d.index
    # y = tptp_d.values
    # plot = ax.plot( x, y,"r*", color=dt_hex )
    # ax.set_xticks( range(2,len(x),10))
    # plt.show()
    # # 折线描点图
    # fig = plt.figure( figsize=(16,4), dpi=100)
    # ax = fig.add_subplot(1,1,1)
    # x = np.array(data2.index)[0:200]
    # y = np.array(data2["USD (PM)"])[0:200]
    # plot = ax.plot( x, y, color=dt_hex, linewidth=2, linestyle='-',label='day' )
    # plt.scatter(temp5_max_index_real[0:12], temp5_max_value[0:12], s=25, c='r') 
    # plt.scatter(temp5_min_index_real[0:12], temp5_min_value[0:12], s=25, c='b') 
    # ax.set_xticks( range(0,len(x),30))
    # plt.xlabel('x',fontsize=20)
    # plt.ylabel('y',fontsize=20)
    # plt.title('title',fontsize=25)
    # ax.legend( loc=0, frameon=True )
    # plt.savefig('天周期点图.png',dpi=600)
    # plt.show()





















    # 5天(一周趋势)
    count = 1
    temp5_max = []
    temp5_min = []
    # temp5_max_index = []
    # temp5_min_index = []
    while True:
        tp_max = max(np.array(data2["USD (PM)"])[(count-1)*short_item:count*short_item])
        tp_min = min(np.array(data2["USD (PM)"])[(count-1)*short_item:count*short_item])
    #     temp5_max_index.append((count-1)*5+list(np.array(data2["USD (PM)"])[(count-1)*5:count*5]).index(tp_max))
    #     temp5_min_index.append((count-1)*5+list(np.array(data2["USD (PM)"])[(count-1)*5:count*5]).index(tp_min))
        temp5_max.append(tp_max)
        temp5_min.append(tp_min)
        count+=1
        if count >= len(data2)/short_item:
            break
    temp5_max_h = amplitude(np.array(temp5_max))
    temp5_min_l = amplitude(np.array(temp5_min))
    #
    tptp = []
    # temp5_max_value = []
    # temp5_min_value = []
    # temp5_max_index_real = []
    # temp5_min_index_real = []
    count = 0
    for i,j in zip(temp5_max_h, temp5_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
    #         temp5_max_value.append(np.array(data2.loc[[temp5_max_index[count]],["USD (PM)"]])[0])
    #         temp5_min_value.append(np.array(data2.loc[[temp5_min_index[count]],["USD (PM)"]])[0])
    #         temp5_max_index_real.append(temp5_max_index[count])
    #         temp5_min_index_real.append(temp5_min_index[count])
        elif i < 0 and j < 0:
            tptp.append(-1)
    #         temp5_max_value.append(np.array(data2.loc[[temp5_max_index[count]],["USD (PM)"]])[0])
    #         temp5_min_value.append(np.array(data2.loc[[temp5_min_index[count]],["USD (PM)"]])[0])
    #         temp5_max_index_real.append(temp5_max_index[count])
    #         temp5_min_index_real.append(temp5_min_index[count])
        else:
            tptp.append(0)
        count += 1
    tptp_5 = pd.DataFrame(tptp)
    #
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = tptp_5.index
#     y = tptp_5.values
#     plot = ax.plot( x, y,"r*", color=dt_hex )
#     ax.set_xticks( range(2,len(x),10))
    # plt.show()














    # 第一个20天
    # 买入-1 卖出+1
    status = 0 # 上一周的涨跌
    #day_amp  黄金日涨幅
    status_dict = [] # (日期，买入还是卖出)
    day = 0
    for i in range(long_item):
        if i == 0: # 第二周
            status_first_5 = np.array(tptp_w_g)[0][0]
            if status_first_5 < 0:
                status = -1 # 跌
            elif status_first_5 > 0:
                status = 1 # 涨
            else:
                status = 0 
        else: # 第三周以后20天以内
            for j in range(short_item):
                day = short_item*i+j+short_item-1
                if status_dict == []: # 没有任何买入时
                    if status == 1:
                        status = np.array(tptp_w_g)[i-1][0]
                        continue
                    elif status == -1 and day_amp[day-1] > 0:
                        status_dict.append((day, -1, np.array(data2)[day][1])) # 买入
                        status = np.array(tptp_w_g)[i-1][0]
                        break
                else: # 有买入后
                    #  上周涨跌判断      本周每日涨跌幅        上笔交易买卖情况   
                    if status == -1 and day_amp[day-1] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                        status_dict.append((day, -1, np.array(data2)[day][1])) # 买入
                        status = np.array(tptp_w_g)[i-1][0]
                        break
                    elif status == -1 and day_amp[day-1] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                        status_dict.append((day, 1, np.array(data2)[day][1])) # 卖出
                        status = np.array(tptp_w_g)[i-1][0]
                        break
    # status_dict









    # # 模拟
    # score = 10000
    # ans = 0 #  手里黄金的盎司
    # for i in status_dict:
    #     ans = ans-(score/(1.01*np.array(data2)[i[0]][1]*i[1]))
    # #     score = score+np.array(data2)[i[0]][1]*i[1] #i[0]*np.array(data2)[i[0]][1]*0.01
    # ans*np.array(data2)[19][1] # 第26天














    # 第二个月以及后面
    for k in range(len(tptp_w_g)):
        if np.array(tptp_w_g)[k][0] == 0:
            status = np.array(tptp_w_g)[k-1][0]
        else:
            status = np.array(tptp_w_g)[k][0]
        for i in range(long_item):
            for j in range(short_item):
                day = long_item*short_item*(k+1)+short_item*i+j
                if status_dict == []: # 前期无交易
                    if status == -1 and day_amp[day-1] > 0: # 上一笔是卖出才能买入或无上一笔可买入
                        status_dict.append((day, -1, np.array(data2)[day][1])) # 买入
                        status = np.array(tptp_w_g)[i-1][0]
                        break
                    elif status == 1 and day_amp[day-1] < 0: # 上一笔是买入才能卖出，无上一笔的不能卖
                        status = np.array(tptp_w_g)[i-1][0]
                        continue
                else: # 前期有交易
                    # 跌幅超过10%交易避免赔本
                    if status == 1 and ((np.array(data2)[day][1]-status_dict[-1][-1])/status_dict[-1][-1]) <= -0.1 and status_dict[-1][1] <= 0:
                        status_dict.append((day, -1, np.array(data2)[day][1])) # 卖出
                        break
                    if status == -1 and day_amp[day-1] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                        status_dict.append((day, -1, np.array(data2)[day][1])) # 买入
                        status = np.array(tptp_w_g)[i-1][0]
                        break
                    elif status == 1 and day_amp[day-1] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                        status_dict.append((day, 1, np.array(data2)[day][1])) # 卖出
                        status = np.array(tptp_w_g)[i-1][0]
                        break
    # status_dict







    # 保存 交易日期 交易情况 交易当日金价
    temp6 = []
    temp6_in_index = []
    temp6_in_value = []
    temp6_out_index = []
    temp6_out_value = []
    for i in status_dict:
        temp6.append([i[0], i[1], np.array(data2)[i[0]][1]])
        if i[1] == -1:
            temp6_in_index.append(i[0])
            temp6_in_value.append(i[2])
        elif i[1] == 1:
            temp6_out_index.append(i[0])
            temp6_out_value.append(i[2])
    output1 = pd.DataFrame(temp6, columns=["Date", "process", "price"])
    # output1









    #### 最终收益 final_zhuan
    temp6_zhuan = []
    # for i,j in zip(temp6_in_value, temp6_out_value):
    #     temp6_zhuan.append(j-i)
    #     print(j-i)
    ans = 0 #  手里黄金盎司
    count = 1
    for i in status_dict:
        if count%2 == 0:
            #print(ans*np.array(data2)[i[0]][1])
            score = (1-a)*ans*np.array(data2)[i[0]][1]
            ans = 0
            temp6_zhuan.append(score)
        else:
            ans = ans-(score/((1+a)*np.array(data2)[i[0]][1]*i[1]))
            score = 0
        count += 1
    final_zhuan = temp6_zhuan[-1]-10000
    if ans != 0:
        final_zhuan += ans*(1-a)*ans*np.array(data2)[-1][1]
    # print("最终收益: {}".format(final_zhuan))
    # 折线描点图
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = np.linspace(0, len(temp6_zhuan), len(temp6_zhuan))
#     y = np.array(temp6_zhuan)
#     plot = ax.plot( x, y, color=dt_hex, linewidth=2, linestyle='-',label='day' )
#     ax.set_xticks( range(0,len(x),100))
#     plt.xlabel('x',fontsize=20)
#     plt.ylabel('y',fontsize=20)
#     plt.title('title',fontsize=25)
#     ax.legend( loc=0, frameon=True )
    # plt.show()
    
    return status_dict,final_zhuan


def bitcoin(short_item, long_item, score, a):
    # 5天(比特币一周趋势)
    count = 1
    temp7_max = []
    temp7_min = []
    # temp7_max_index = []
    # temp7_min_index = []
    while True:
        tp_max = max(np.array(data1["Value"])[(count-1)*short_item:count*short_item])
        tp_min = min(np.array(data1["Value"])[(count-1)*short_item:count*short_item])
    #     temp7_max_index.append((count-1)*7+list(np.array(data1["Value])[(count-1)*7:count*7]).index(tp_max))
    #     temp7_min_index.append((count-1)*7+list(np.array(data1["Value"])[(count-1)*7:count*7]).index(tp_min))
        temp7_max.append(tp_max)
        temp7_min.append(tp_min)
        count+=1
        if count >= len(data1)/short_item:
            break
    temp7_max_h = amplitude(np.array(temp7_max))
    temp7_min_l = amplitude(np.array(temp7_min))
    #
    tptp = []
    # temp7_max_value = []
    # temp7_min_value = []
    # temp7_max_index_real = []
    # temp7_min_index_real = []
    count = 0
    for i,j in zip(temp7_max_h, temp7_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
    #         temp7_max_value.append(np.array(data1.loc[[temp7_max_index[count]],["Value"]])[0])
    #         temp7_min_value.append(np.array(data1.loc[[temp7_min_index[count]],[["Value"]])[0])
    #         temp7_max_index_real.append(temp7_max_index[count])
    #         temp7_min_index_real.append(temp7_min_index[count])
        elif i < 0 and j < 0:
            tptp.append(-1)
    #         temp7_max_value.append(np.array(data1.loc[[temp7_max_index[count]],["Value"]])[0])
    #         temp7_min_value.append(np.array(data1.loc[[temp7_min_index[count]],["Value"]])[0])
    #         temp7_max_index_real.append(temp7_max_index[count])
    #         temp7_min_index_real.append(temp7_min_index[count])
        else:
            tptp.append(0)
        count += 1
    tptp_7 = pd.DataFrame(tptp)
    #
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = tptp_7.index
#     y = tptp_7.values
#     plot = ax.plot( x, y,"r*", color=dt_hex )
#     ax.set_xticks( range(2,len(x),10))
    # plt.show()








    # 周比特币平均值
    count = 1
    temp8 = []
    while True:
        tp = data1.iloc[short_item*(count-1): short_item*count]["Value"].mean()
        count+= 1
        temp8.append(tp)
        if count >= len(data1)/short_item:
            break
    temp8_df = pd.DataFrame(temp8) # 252周的比特币价
    # 周平均比特币图
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = temp8_df.index
#     y = temp8_df.values
#     plot = ax.plot( x, y, color=dt_hex, linewidth=2, linestyle='-',label='week' )
#     ax.set_xticks( range(0,len(x),10))
#     ax.legend( loc=0, frameon=True )
    # plt.show()
    # 长期(28天)
    count = 1
    temp8_max = [] # 周高位点
    temp8_min = [] # 周低位点
    while True:
        tp_max = max(temp8[(count-1)*long_item:count*long_item])
        tp_min = min(temp8[(count-1)*long_item:count*long_item])
        temp8_max.append(tp_max)
        temp8_min.append(tp_min)
        count+=1
        if count >= len(temp8)/long_item:
            break
    temp8_max_h = amplitude(np.array(temp8_max)) # 周涨幅
    temp8_min_l = amplitude(np.array(temp8_min)) # 周跌幅
    #周趋势(同增通减同趋势，不同则未知)
    tptp = []
    for i,j in zip(temp8_max_h, temp8_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
        elif i < 0 and j < 0:
            tptp.append(-1)
        else:
            tptp.append(0)
    tptp_w_b = pd.DataFrame(tptp) # 第二周开始的61周涨跌幅度表 -1跌+1涨
    #周趋势图(长期)
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = tptp_w_b.index
#     y = tptp_w_b.values
#     plot = ax.plot( x, y,"r*", color=dt_hex )
#     ax.set_xticks( range(2,len(x),10))
    # ax.legend( loc=0, frameon=True )
    # plt.show()










    # 第一个28天
    # 买入-1 卖出+1
    status = 0 # 上一周的涨跌
    status_dict = [] # (日期，买入还是卖出)
    day = 0
    for i in range(long_item):
        if i == 0: # 第二周
            status_first_7 = np.array(tptp_w_b)[0][0]
            if status_first_7 < 0:
                status = -1 # 跌
            elif status_first_7 > 0:
                status = 1 # 涨
            else:
                status = 0 
        else: # 第三周以后28天以内
            for j in range(short_item):
                day = short_item*i+j+short_item-1
                if status_dict == []: # 没有任何买入时
                    if status == 1:
                        status = np.array(tptp_w_b)[i-1][0]
                        continue
                    elif status == -1 and day_b_amp[day] > 0:
                        status_dict.append((day, -1, np.array(data1)[day][1])) # 买入
                        status = np.array(tptp_w_b)[i-1][0]
                        break
                else: # 有买入后
                    #  上周涨跌判断      本周每日涨跌幅        上笔交易买卖情况   
                    if status == -1 and day_b_amp[day] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                        status_dict.append((day, -1, np.array(data1)[day][1])) # 买入
                        status = np.array(tptp_w_b)[i-1][0]
                        break
                    elif status == -1 and day_b_amp[day] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                        status_dict.append((day, 1, np.array(data1)[day][1])) # 卖出
                        status = np.array(tptp_w_b)[i-1][0]
                        break
    # status_dict












    # 第二个月以及后面
    for k in range(len(tptp_w_b)):
        if np.array(tptp_w_b)[k][0] == 0:
            status = np.array(tptp_w_b)[k-1][0]
        else:
            status = np.array(tptp_w_b)[k][0]
        for i in range(long_item):
            for j in range(short_item):
                day = short_item*long_item*(k+1)+short_item*i+j
                if status_dict == []:# 前期无交易
                    if status == -1 and day_b_amp[day] > 0: # 上一笔是卖出才能买入或无上一笔可买入
                        status_dict.append((day, -1, np.array(data1)[day][1])) # 买入
                        status = np.array(tptp_w_b)[i-1][0]
                        break
                    elif status == 1 and day_b_amp[day] < 0: # 上一笔是买入才能卖出，无上一笔的不能卖
                        status = np.array(tptp_w_b)[i-1][0]
                        continue
                else:# 前期有交易
                    # 跌幅超过10%交易避免赔本
                    if status == -1 and ((np.array(data1)[day][1]-status_dict[-1][-1])/status_dict[-1][-1]) <= -0.1 and status_dict[-1][1] <= 0:
                        status_dict.append((day, 1, np.array(data1)[day][1])) # 卖出
                        continue
                    if status == -1 and day_b_amp[day] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                        status_dict.append((day, -1, np.array(data1)[day][1])) # 买入
                        status = np.array(tptp_w_b)[i-1][0]
                        break
                    elif status == -1 and day_b_amp[day] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                        status_dict.append((day, 1, np.array(data1)[day][1])) # 卖出
                        status = np.array(tptp_w_b)[i-1][0]
                        break

    # status_dict









    # 保存 交易日期 交易情况 交易当日金价
    temp9 = []
    temp9_in_index = []
    temp9_in_value = []
    temp9_out_index = []
    temp9_out_value = []
    for i in status_dict:
        temp9.append([i[0], i[1], np.array(data1)[i[0]][1]])
        if i[1] == -1:
            temp9_in_index.append(i[0])
            temp9_in_value.append(i[2])
        elif i[1] == 1:
            temp9_out_index.append(i[0])
            temp9_out_value.append(i[2])
    output1 = pd.DataFrame(temp9, columns=["Date", "process", "price"])
    # output1









    ### 收益计算 final_zhuan
    temp9_zhuan = []
    # for i,j in zip(temp9_in_value, temp9_out_value):
    #     temp9_zhuan.append(j-i)
    #     print(j-i)
    # 模拟本金
    ans = 0 #  手里比特币颗数
    count = 1
    for i in status_dict:
        if count%2 == 0:
            #print(ans*np.array(data1)[i[0]][1])
            score = (1-a)*ans*np.array(data1)[i[0]][1]
            ans = 0
            temp9_zhuan.append(score)
        else:
            ans = ans-(score/((1+a)*np.array(data1)[i[0]][1]*i[1]))
            score = 0
        count += 1
    final_zhuan = temp9_zhuan[-1]-10000
    if ans != 0:
        final_zhuan += (1-a)*ans*np.array(data1)[-1][1]
    # print("最终收益: {}".format(final_zhuan))
    # 折线描点图
#     fig = plt.figure( figsize=(16,4), dpi=100)
#     ax = fig.add_subplot(1,1,1)
#     x = np.linspace(0, len(temp9_zhuan), len(temp9_zhuan))
#     y = np.array(temp9_zhuan)
#     plot = ax.plot( x, y, color=dt_hex, linewidth=2, linestyle='-',label='day' )
#     ax.set_xticks( range(0,len(x),100))
#     plt.xlabel('x',fontsize=20)
#     plt.ylabel('y',fontsize=20)
#     plt.title('title',fontsize=25)
#     ax.legend( loc=0, frameon=True )
    # plt.show()
    return status_dict, final_zhuan



def Rt(Pt):
    Rt = []
    p1 = np.array(Pt.copy()[1:])
    p2 = np.array(Pt.copy()[:-1])
    count = 0
    for i,j in zip(p1, p2):
        count+=1
        rt = 0
        if count <= 60:
            for k in range(count):
                rt = rt + (p1[count-1]-p2[count-1])/p2[count-1]
        else:
            for k in range(60):
                rt = rt + (p1[count-k-1]-p2[count-k-1])/p2[count-k-1]
        Rt.append(rt)
    return Rt

def ERt(Rt):
    ERt = []
    n = len(Rt)
    for i in range(1,n+1):
        ERt.append(np.array(list(Rt[0:i])).sum()/i)
    return ERt

def Dt(Rt):
    DRt = []
    n = len(Rt)
    count = 0
    for i in range(1,n+1):
#         tpd = 0
#         for j in range(1,i):
#             tpd = tpd+(Rt[j]-ERt(Rt[0:i])[-1])**2
#         DRt.append(tpd/(i-1))
        count += 1
        if count <= 60:
            DRt.append(np.std(Rt[0:i],ddof = 1))
        else:
            DRt.append(np.std(Rt[i-60:i],ddof = 1))
    return DRt









data_aa = []
for a1 in [0.01, 0.01, 0.05, 0.1]:
    data_a2 = []
    for a2 in [0.001, 0.01, 0.05, 0.1]:
        try:
            # gold(5,4)
            temp_zz = []
            for i in range(5, 20): # 中期
                for j in range(2, 5): # 多少中期为长期
                    temp_zz.append((i,j,gold(i, j, 1000, a1)))
            temp_zzz = 0
            for i in temp_zz:
                if temp_zzz > i[2][-1]:
                    continue
                elif temp_zzz <= i[2][-1]:
                    temp_zzz = i[2][-1]
                    tp_z = i
                else:
                    tp_z = 0

            gold_tp_z = tp_z.copy()

            # bitcoin(7,4)
            temp_zz = []
            for i in range(5, 20): # 中期
                for j in range(2, 5): # 多少中期为长期
                    temp_zz.append((i,j,bitcoin(i, j, 1000, a2)))
            temp_zzz = 0
            for i in temp_zz:
                if temp_zzz > i[2][-1]:
                    continue
                elif temp_zzz <= i[2][-1]:
                    temp_zzz = i[2][-1]
                    tp_z = i
                else:
                    tp_z = 0

            bit_tp_z = tp_z.copy()


            # 先向前取值填充，再先后取值填充
            zdata = pd.merge(data1, data2, how='outer').fillna(method='ffill').fillna(method='backfill')


            Rgt = Rt(zdata["USD (PM)"])
            ERgt = ERt(Rgt)
            DRgt = Dt(Rgt)
            Rbt = Rt(zdata["Value"])
            ERbt = ERt(Rbt)
            DRbt = Dt(Rbt)


            #                          金R   币R  金标  币标
            tpp = []
            m_list = []
            for m in [(i+1)/10 for i in range(10)]:
                tp = []
                for rg,rb,dg,db in zip(Rgt[1:], Rbt[1:], DRgt[1:], DRbt[1:]):
                    #rx = (rg-0.01)*x1+(rb-0.02)*x2
                    #dg*x1+db*x2<=0.22
                    #(1+0.01)*x1+(1+0.02)*x2=1
                    #x1,x2>0
                #     c = np.array([-(rg-0.01), -(rb-0.02)])
                    A_ub = np.array([[dg, db]])  # 不等式约束
                    b_ub = np.array([0.22])
                    c = np.array([-m*(rg-a1)+(1-m)*dg, -m*(rb-a2)+(1-m)*db])
                    A_eq = np.array([[1+a1, 1+a2]])                # 等式约束
                    b_eq = np.array([1])
                    r = linprog(c , A_ub, b_ub, A_eq, b_eq, bounds=((0, None), (0, None)))#, method='simplex'#
                    if dict(r)['success'] == True:
            #             print(dict(r)['fun'],dict(r)['x'])
                        tp.append(dict(r)['x'])
                    else:
                        print("error")
                pd.DataFrame(tp).to_csv("temp.csv")

                # 计算收益
                np.set_printoptions(suppress=True)
                B = pd.read_csv(r'B.csv') # B
                H = pd.read_csv(r'H.csv') # H
                Times = pd.read_csv(r'temp.csv') # Time
                B = B.set_index("Unnamed: 0")
                H = H.set_index("Unnamed: 0")
                Times = Times.set_index("Unnamed: 0")
                # 先向前取值填充，再先后取值填充
                BH = pd.merge(H.iloc[:,0:2], B.iloc[:,0:2], how='outer',on='0').sort_values('0',ascending=True)
                BH = pd.merge(BH, H.iloc[:,0:3:2], how="left", on=["0"])
                BH = pd.merge(BH, B.iloc[:,0:3:2], how="left", on=["0"]).fillna(0)
                m = 1000
                h = 0
                b = 0
                p = 0
                q = 0
                control_list = []
                for i in range(len(Times)):
                    x1 = 0
                    x2 = 0
                    j = 0
                    t = 0
                    if i <= 9:
                        control_list.append([i,0,m,h,j,b,t,x1,x2])
                        continue
                    else:
                        if i not in list(np.array(BH['0'])):
                            control_list.append([i,0,m,h,j,b,t,x1,x2])
                            continue
                    if np.array(BH[BH['0'].isin([str(i)])]['1_x'])[0] != 0: # H
                #         if np.array(BH[BH['0'].isin([str(i)])]['2_y'])[0] == 0: # B
                        if np.array(BH[BH['0'].isin([str(i)])]['1_x'])[0] < 0: # 买
                            j = np.array(H[H['0'].isin([i])]['2'])[0]
                            x1 = np.array(Times.iloc[[i]]['0'])[0]
                            x2 = np.array(Times.iloc[[i]]['1'])[0]
                            p = m*x1#(x1/(x1+x2))
                            q = m*x2#(x2/(x1+x2))
                            h = (p-0.01*p)/j
                            p = 0
                            m = p + q
                            control_list.append([i,11,m,h,j,b,0,x1,x2])
                #             print(m)
                        if np.array(BH[BH['0'].isin([str(i)])]['1_x'])[0] > 0: # 卖
                            j = np.array(H[H['0'].isin([i])]['2'])[0]
                            x1 = np.array(Times.iloc[[i]]['0'])[0]
                            x2 = np.array(Times.iloc[[i]]['1'])[0]
                            p = m*x1
                            q = m*x2
                            m = h*j-h*j*0.01+p+q
                            h = 0
                            control_list.append([i,-11,m,h,j,b,0,x1,x2])
                #             print(m)

                #             if np.array(BH[BH['0'].isin([str(i)])]['2_x'])[0] == 0: # H
                    if np.array(BH[BH['0'].isin([str(i)])]['1_y'])[0] != 0: # B
                        if np.array(BH[BH['0'].isin([str(i)])]['1_y'])[0] < 0: # 买
                            t = np.array(B[B['0'].isin([i])]['2'])[0]
                            x1 = np.array(Times.iloc[[i]]['0'])[0]
                            x2 = np.array(Times.iloc[[i]]['1'])[0]
                            p = m*x1#(x1/(x1+x2))
                            q = m*x2#(x2/(x1+x2))
                            b = (q-0.02*q)/t
                            q = 0
                            m = p + q
                            control_list.append([i,22,m,h,0,b,t,x1,x2])
                #             print(m)
                        elif np.array(BH[BH['0'].isin([str(i)])]['1_y'])[0] > 0: # 卖
                            t = np.array(B[B['0'].isin([i])]['2'])[0]
                            x1 = np.array(Times.iloc[[i]]['0'])[0]
                            x2 = np.array(Times.iloc[[i]]['1'])[0]
                            p = m*x1
                            q = m*x2
                            m = b*t-b*t*0.02+p+q
                            b = 0
                            control_list.append([i,-22,m,h,0,b,t,x1,x2])
                print(m)
                m_list.append(m)
                tpp.append(tp)
                if m == max(m_list):
                    cp = tp.copy()
            print(cp)
            data_a2.append(cp)
        except:
            data_a2.append(0)
    data_aa.append(data_a2)